# Disease Prediction using Machine Learning Algorithms

### Importing Libraries

In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix ,classification_report


### Loading Data

#### Main Dataset

In [26]:
df = pd.read_csv('../dataset/dataset.csv')
print(df.head())


            Disease   Symptom_1              Symptom_2              Symptom_3  \
0  Fungal infection     itching              skin_rash   nodal_skin_eruptions   
1  Fungal infection   skin_rash   nodal_skin_eruptions    dischromic _patches   
2  Fungal infection     itching   nodal_skin_eruptions    dischromic _patches   
3  Fungal infection     itching              skin_rash    dischromic _patches   
4  Fungal infection     itching              skin_rash   nodal_skin_eruptions   

              Symptom_4 Symptom_5 Symptom_6 Symptom_7 Symptom_8 Symptom_9  \
0   dischromic _patches       NaN       NaN       NaN       NaN       NaN   
1                   NaN       NaN       NaN       NaN       NaN       NaN   
2                   NaN       NaN       NaN       NaN       NaN       NaN   
3                   NaN       NaN       NaN       NaN       NaN       NaN   
4                   NaN       NaN       NaN       NaN       NaN       NaN   

  Symptom_10 Symptom_11 Symptom_12 Symptom_13 Symp

#### Symptom Severity Dataset

In [27]:
df1 = pd.read_csv('../dataset/Symptom-severity.csv')
print(df1.head())

                Symptom  weight
0               itching       1
1             skin_rash       3
2  nodal_skin_eruptions       4
3   continuous_sneezing       4
4             shivering       5


#### Symptom Description Dataset

In [28]:
df2 = pd.read_csv('../disease-symptom-description-dataset/symptom_Description.csv')
print(df2.head())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\665669761.py", line 1, in <module>
    df2 = pd.read_csv('../disease-symptom-description-dataset/symptom_Description.csv')
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\pandas\util\_decorators.py", line 211, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\pandas\util\_decorators.py", line 331, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\pandas\io\parsers\readers.py", line 950, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Throug

#### Symptom Precaution Dataset

In [29]:
df3 = pd.read_csv('../dataset/symptom_precaution.csv')
print(df3.head())

          Disease                      Precaution_1  \
0   Drug Reaction                   stop irritation   
1         Malaria          Consult nearest hospital   
2         Allergy                    apply calamine   
3  Hypothyroidism                     reduce stress   
4       Psoriasis  wash hands with warm soapy water   

                   Precaution_2        Precaution_3  \
0      consult nearest hospital    stop taking drug   
1               avoid oily food  avoid non veg food   
2       cover area with bandage                 NaN   
3                      exercise         eat healthy   
4  stop bleeding using pressure      consult doctor   

                  Precaution_4  
0                    follow up  
1           keep mosquitos out  
2  use ice to compress itching  
3             get proper sleep  
4                   salt baths  


#### List of Diseases

In [30]:
diseases = set(df['Disease'])

print(diseases)
print('\n There are {} diseases described in the dataset'.format(len(diseases)))



{'Migraine', 'Hypoglycemia', 'Hepatitis D', 'Pneumonia', 'Malaria', 'Hepatitis E', 'Urinary tract infection', 'Impetigo', 'Arthritis', 'Jaundice', 'Allergy', 'Drug Reaction', 'Fungal infection', 'Hypertension ', 'Paralysis (brain hemorrhage)', 'Alcoholic hepatitis', 'Dimorphic hemmorhoids(piles)', 'Gastroenteritis', 'Hypothyroidism', 'Peptic ulcer diseae', 'Heart attack', 'Acne', 'Hyperthyroidism', 'Psoriasis', 'Cervical spondylosis', 'Diabetes ', 'AIDS', 'Tuberculosis', 'Chicken pox', 'Chronic cholestasis', 'Common Cold', 'Osteoarthristis', 'GERD', 'Hepatitis C', 'Bronchial Asthma', '(vertigo) Paroymsal  Positional Vertigo', 'Dengue', 'Typhoid', 'hepatitis A', 'Hepatitis B', 'Varicose veins'}

 There are 41 diseases described in the dataset


#### List of symptoms

In [31]:
symptoms = set(df.iloc[:,1:].values.flatten())

print(symptoms)
print('\nThere are {} symptoms described in the dataset'.format(len(symptoms) - 1))

{nan, ' constipation', ' chills', ' burning_micturition', ' high_fever', ' irritability', ' yellowish_skin', ' cough', ' swelling_joints', ' painful_walking', ' puffy_face_and_eyes', ' runny_nose', ' diarrhoea', ' small_dents_in_nails', ' sunken_eyes', ' red_spots_over_body', ' slurred_speech', ' skin_rash', ' extra_marital_contacts', ' swollen_legs', ' pus_filled_pimples', ' enlarged_thyroid', ' lack_of_concentration', ' anxiety', ' stomach_bleeding', ' mild_fever', ' bladder_discomfort', ' pain_during_bowel_movements', ' red_sore_around_nose', ' obesity', ' pain_behind_the_eyes', ' weakness_in_limbs', ' ulcers_on_tongue', ' fluid_overload', ' blood_in_sputum', ' excessive_hunger', ' sinus_pressure', ' unsteadiness', ' breathlessness', ' muscle_pain', ' brittle_nails', 'itching', ' spinning_movements', ' altered_sensorium', ' nodal_skin_eruptions', ' blackheads', ' blister', ' hip_joint_pain', ' acidity', ' shivering', ' family_history', ' belly_pain', ' pain_in_anal_region', ' malais

### Data Preprocessing

##### Handling Missing data

In [32]:
sns.heatmap(df.isnull(), cmap='Blues')
plt.title('Null Values Heatmap')
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\384121996.py", line 1, in <module>
    sns.heatmap(df.isnull(), cmap='Blues')
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  F

In [33]:
df.isnull().sum()


Disease          0
Symptom_1        0
Symptom_2        0
Symptom_3        0
Symptom_4      348
Symptom_5     1206
Symptom_6     1986
Symptom_7     2652
Symptom_8     2976
Symptom_9     3228
Symptom_10    3408
Symptom_11    3726
Symptom_12    4176
Symptom_13    4416
Symptom_14    4614
Symptom_15    4680
Symptom_16    4728
Symptom_17    4848
dtype: int64

In [34]:
df = df.apply(lambda x: x.str.strip() )
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x: x.str.lower() )
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x: x.str.replace(' ', ''))


df = df.fillna(0)

symptoms = list(set(df.iloc[:,1:].values.flatten()))
symptoms.remove(0)

df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic_patches,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic_patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,itching,nodal_skin_eruptions,dischromic_patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,itching,skin_rash,dischromic_patches,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,vomiting,headache,nausea,spinning_movements,loss_of_balance,unsteadiness,0,0,0,0,0,0,0,0,0,0,0
4916,Acne,skin_rash,pus_filled_pimples,blackheads,scurring,0,0,0,0,0,0,0,0,0,0,0,0,0
4917,Urinary tract infection,burning_micturition,bladder_discomfort,foul_smell_ofurine,continuous_feel_of_urine,0,0,0,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,skin_rash,joint_pain,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,0,0,0,0,0,0,0,0,0,0,0


##### Encoding  the symptoms with their severity weight

In [35]:
# Create a dictionary mapping each symptom to its severity
severity_map = dict(zip(df1['Symptom'], df1['weight']))

     

# replace the symptoms  with their severity
encoded_df = df.replace(severity_map)

encoded_df

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,1,3,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,3,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,4,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,(vertigo) Paroymsal Positional Vertigo,5,3,5,6,4,4,0,0,0,0,0,0,0,0,0,0,0
4916,Acne,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0
4917,Urinary tract infection,6,4,5,6,0,0,0,0,0,0,0,0,0,0,0,0,0
4918,Psoriasis,3,3,3,2,2,2,0,0,0,0,0,0,0,0,0,0,0


#### Storing the diseases and encoded symptoms in seperate dataframes

In [36]:
cols = df.columns

(encoded_df[cols] == 0).all()

encoded_df['Disease'].value_counts()

encoded_df['Disease'].unique()

data = encoded_df.iloc[:,1:].values  
labels = encoded_df['Disease'].values 



#### Splitting the data 


In [37]:
x_train, x_test, y_train, y_test = train_test_split(data, labels,  train_size = 0.70, shuffle=True, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(3444, 17) (1476, 17) (3444,) (1476,)


### Model Training and Testing

####  Random Forest Classifier

In [38]:
rfc = RandomForestClassifier()
rfc.fit(x_train, y_train)
result = rfc.predict(x_test)

conf_mat = confusion_matrix(y_test, result)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())

sns.heatmap(df_cm)
print(conf_mat)

rfc_score = accuracy_score(y_test, result) * 100

print('Accuracy% =', rfc_score)
print('F1 Score =', f1_score(y_test, result, average='weighted') *  100)



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\1823417516.py", line 8, in <module>
    sns.heatmap(df_cm)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Do

#### Decision Tree Classifier

In [39]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
result = dtc.predict(x_test)

conf_mat = confusion_matrix(y_test, result)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())

sns.heatmap(df_cm)

dtc_score = accuracy_score(y_test, result) * 100

print( 'Accuracy% =', dtc_score)
print('F1 Score =', f1_score(y_test, result, average='weighted') * 100)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\3419466785.py", line 8, in <module>
    sns.heatmap(df_cm)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Do

#### Naives Bayes Classifier

In [40]:
nbc = GaussianNB()
nbc.fit(x_train, y_train)
result = nbc.predict(x_test)

conf_mat = confusion_matrix(y_test, result)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())

sns.heatmap(df_cm)

nbc_score = accuracy_score(y_test, result) * 100

print( 'Accuracy% =', nbc_score)
print('F1 Score =', f1_score(y_test, result, average='weighted') * 100)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\2342768387.py", line 8, in <module>
    sns.heatmap(df_cm)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Do

#### K-Nearest Neighbors Classifier

In [41]:
knc = KNeighborsClassifier()
knc.fit(x_train, y_train)
result = knc.predict(x_test)

conf_mat = confusion_matrix(y_test, result)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())

sns.heatmap(df_cm)

knc_score = accuracy_score(y_test, result) * 100

print( 'Accuracy% =', knc_score)
print('F1 Score =', f1_score(y_test, result, average='weighted') * 100)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\4150872270.py", line 8, in <module>
    sns.heatmap(df_cm)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Do

#### Support Vector Classifier

In [42]:
svc = SVC(probability=True)
svc.fit(x_train, y_train)
result = svc.predict(x_test)

conf_mat = confusion_matrix(y_test, result)
df_cm = pd.DataFrame(conf_mat, index=df['Disease'].unique(), columns=df['Disease'].unique())

sns.heatmap(df_cm)

svc_score = accuracy_score(y_test, result) * 100

print( 'Accuracy% =', svc_score)
print('F1 Score =', f1_score(y_test, result, average='weighted') * 100)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\1782031249.py", line 8, in <module>
    sns.heatmap(df_cm)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\matrix.py", line 456, in heatmap
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\Hp\Do

#### Comparing the accuracy of all the models

In [43]:
models = ['Random Forest', 'Decision Tree', 'Naive Bayes', 'KNN', 'SVM']
accuracies = [rfc_score, dtc_score, nbc_score, knc_score, svc_score]

sns.set_style("whitegrid")
sns.set_palette("Set1")
sns.barplot(x=models, y=accuracies)

plt.ylim(85, 100)
plt.xlabel("Machine Learning Models")
plt.ylabel("Accuracy")
plt.title("Comparison of Accuracy between Different ML Models")
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\2599035756.py", line 2, in <module>
    accuracies = [rfc_score, dtc_score, nbc_score, knc_score, svc_score]
NameError: name 'rfc_score' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\pygments\styles\__init__.py", line 82, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 2057, in showtraceback
  Fi

### Making Predictions

In [44]:
def predict_disease(symptoms_list,model,top_k=5):
    # Encode the symptoms based on their severity
    encoded_symptoms = [severity_map[symptom] for symptom in symptoms_list]
    
    for i  in range(len(encoded_symptoms),17):
        encoded_symptoms.append(0)
        
    # Create a numpy array from the encoded symptoms
    symptoms_array = np.array(encoded_symptoms).reshape(1,-1)
 
     # Use the trained model to predict the probabilities of all diseases
    disease_probs = model.predict_proba(symptoms_array)[0]
    
    # Sort the predicted probabilities in descending order
    sorted_probs_idx = np.argsort(disease_probs)[::-1]
    
    # Get the top-k predicted diseases and their probabilities
    top_k_diseases = [(model.classes_[idx], disease_probs[idx]) for idx in sorted_probs_idx[:top_k]]
    
    return top_k_diseases

In [45]:
def output_predicted_diseases(symptoms_list,model,top_k=5):
    top_k_diseases = predict_disease(symptoms_list,model,top_k)
    
    print('The predicted disease is {} with probability {:.2f}%\n'.format(top_k_diseases[0][0], top_k_diseases[0][1]*100))
    
    print(top_k_diseases)

    predict_diseases = [disease[0] for disease in top_k_diseases]
    
    predict_probs = [disease[1] for disease in top_k_diseases]
    
    sns.set_style("whitegrid")
    sns.set_palette("Set1")
    sns.barplot(x=predict_diseases, y=predict_probs)
    

    plt.xlabel("Different Diseases")
    plt.ylabel("Probability")
    plt.title("Predicted Diseases and their Probabilities")
    plt.tight_layout()
    plt.show()


In [46]:
symptomsList = ['itching', 'skin_peeling', 'skin_rash']

output_predicted_diseases(symptomsList,rfc)
      

The predicted disease is Fungal infection with probability 68.00%

[('Fungal infection', 0.68), ('Psoriasis', 0.11), ('Paralysis (brain hemorrhage)', 0.11), ('Acne', 0.04), ('Allergy', 0.02)]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\692808414.py", line 3, in <module>
    output_predicted_diseases(symptomsList,rfc)
  File "C:\Users\Hp\AppData\Local\Temp\ipykernel_10416\312904620.py", line 14, in output_predicted_diseases
    sns.barplot(x=predict_diseases, y=predict_probs)
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\seaborn\categorical.py", line 2761, in barplot
    ax = plt.gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 2309, in gca
    return gcf().gca()
  File "c:\Users\Hp\Documents\ML-Projects\Disease-Prediction-Through-Symptoms\env\lib\site-packages\matplotlib\pyplot.py", line 906, in gcf
    return figure()
  File "c:\Users\H

### Deploying Trained Model

In [47]:
with open('../models/RandomForestClassifier.plk', 'wb') as file:
    pickle.dump(rfc, file)
    
with open('../models/DecisionTreeClassifier.plk', 'wb') as file:
      pickle.dump(dtc, file)
      
with open('../models/GaussianNB.plk', 'wb') as file:
      pickle.dump(nbc, file)

with open('../models/KNeighborsClassifier.plk', 'wb') as file:
      pickle.dump(knc, file)
      
with open('../models/SVC.plk', 'wb') as file:
      pickle.dump(svc, file)
      
with open('../models/symptoms.plk', 'wb') as file:
      pickle.dump(symptoms, file)
      
with open('../models/severity_map.plk', 'wb') as f:
    pickle.dump(severity_map, f)
    
precaution_dict = {}    
    
for row in df3.values:
    precaution_dict[row[0]] = row[1:]   

with open('../models/precaution_dict.plk', 'wb') as f:
    pickle.dump(precaution_dict, f)      